In [84]:
!pip install prophet
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [110]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings
import pickle

In [86]:
warnings.filterwarnings('ignore')

In [87]:
pd.options.display.float_format = '${:,.2f}'.format

In [88]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2018-06-22'

In [89]:
df = yf.download('BTC-USD', start_date, today)

[*********************100%***********************]  1 of 1 completed


In [90]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-06-22,"$6,737.88","$6,747.08","$6,006.60","$6,083.69","$6,083.69",5079810048
2018-06-23,"$6,090.10","$6,224.82","$6,071.81","$6,162.48","$6,162.48",3431360000
2018-06-24,"$6,164.28","$6,223.78","$5,826.41","$6,173.23","$6,173.23",4566909952
2018-06-25,"$6,171.97","$6,327.37","$6,119.68","$6,249.18","$6,249.18",5500810240
2018-06-26,"$6,253.55","$6,290.16","$6,093.67","$6,093.67","$6,093.67",3279759872
...,...,...,...,...,...,...
2023-06-18,"$26,510.46","$26,675.93","$26,325.89","$26,336.21","$26,336.21",9565695129
2023-06-19,"$26,335.44","$26,984.61","$26,312.83","$26,851.03","$26,851.03",12826986222
2023-06-20,"$26,841.66","$28,388.97","$26,668.79","$28,327.49","$28,327.49",22211859147


In [91]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-06-22,"$6,737.88","$6,747.08","$6,006.60","$6,083.69","$6,083.69",5079810048
2018-06-23,"$6,090.10","$6,224.82","$6,071.81","$6,162.48","$6,162.48",3431360000
2018-06-24,"$6,164.28","$6,223.78","$5,826.41","$6,173.23","$6,173.23",4566909952
2018-06-25,"$6,171.97","$6,327.37","$6,119.68","$6,249.18","$6,249.18",5500810240
2018-06-26,"$6,253.55","$6,290.16","$6,093.67","$6,093.67","$6,093.67",3279759872


In [92]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-18,"$26,510.46","$26,675.93","$26,325.89","$26,336.21","$26,336.21",9565695129
2023-06-19,"$26,335.44","$26,984.61","$26,312.83","$26,851.03","$26,851.03",12826986222
2023-06-20,"$26,841.66","$28,388.97","$26,668.79","$28,327.49","$28,327.49",22211859147
2023-06-21,"$28,311.31","$30,737.33","$28,283.41","$30,027.30","$30,027.30",33346760979
2023-06-22,"$29,995.94","$30,496.00","$29,679.16","$29,912.28","$29,912.28",20653160491


In [93]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,"$1,827.00","$1,827.00","$1,827.00","$1,827.00","$1,827.00","$1,827.00"
mean,"$21,829.76","$22,347.73","$21,263.75","$21,840.32","$21,840.32","$27,829,804,145.10"
std,"$16,667.44","$17,103.62","$16,158.52","$16,660.30","$16,660.30","$19,165,001,767.22"
min,"$3,236.27","$3,275.38","$3,191.30","$3,236.76","$3,236.76","$2,923,670,016.00"
25%,"$8,165.63","$8,270.12","$7,962.39","$8,173.52","$8,173.52","$15,868,028,487.00"
50%,"$16,836.47","$16,975.02","$16,614.03","$16,841.99","$16,841.99","$25,370,975,378.00"
75%,"$33,767.38","$34,778.08","$32,265.80","$33,734.52","$33,734.52","$36,137,652,176.00"
max,"$67,549.73","$68,789.62","$66,382.06","$67,566.83","$67,566.83","$350,967,941,479.00"


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1827 entries, 2018-06-22 to 2023-06-22
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1827 non-null   float64
 1   High       1827 non-null   float64
 2   Low        1827 non-null   float64
 3   Close      1827 non-null   float64
 4   Adj Close  1827 non-null   float64
 5   Volume     1827 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 99.9 KB


In [95]:
df.isnull().any()

Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
dtype: bool

In [96]:
df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [97]:
df.reset_index(inplace=True)
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [98]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-06-22,"$6,737.88","$6,747.08","$6,006.60","$6,083.69","$6,083.69",5079810048
1,2018-06-23,"$6,090.10","$6,224.82","$6,071.81","$6,162.48","$6,162.48",3431360000
2,2018-06-24,"$6,164.28","$6,223.78","$5,826.41","$6,173.23","$6,173.23",4566909952
3,2018-06-25,"$6,171.97","$6,327.37","$6,119.68","$6,249.18","$6,249.18",5500810240
4,2018-06-26,"$6,253.55","$6,290.16","$6,093.67","$6,093.67","$6,093.67",3279759872


In [99]:
df1 = df[["Date", "Open"]]
df1.head()

,Date,Open
0,2018-06-22,"$6,737.88"
1,2018-06-23,"$6,090.10"
2,2018-06-24,"$6,164.28"
3,2018-06-25,"$6,171.97"
4,2018-06-26,"$6,253.55"


In [100]:
new_names = {"Date": "ds", "Open": "y"}
df1.rename(columns = new_names, inplace=True)

In [101]:
df1.head()

,ds,y
0,2018-06-22,"$6,737.88"
1,2018-06-23,"$6,090.10"
2,2018-06-24,"$6,164.28"
3,2018-06-25,"$6,171.97"
4,2018-06-26,"$6,253.55"


In [102]:
x = df1["ds"]
y = df1["y"]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.update_layout(title_text = "Time Series Plot of Bitcoin Open Price")

fig.update_layout(
    xaxis = dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ]
            )
        ),
        rangeslider = dict(visible=True),
        type="date"
    )
)

In [103]:
m = Prophet(seasonality_mode="multiplicative")
m.fit(df1)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp261v31ir/7immwviz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp261v31ir/smzh2bxr.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=10630', 'data', 'file=/tmp/tmp261v31ir/7immwviz.json', 'init=/tmp/tmp261v31ir/smzh2bxr.json', 'output', 'file=/tmp/tmp261v31ir/prophet_model6mh9rq29/prophet_model-20230623135036.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:50:36 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:50:38 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [104]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2187,2024-06-17
2188,2024-06-18
2189,2024-06-19
2190,2024-06-20
2191,2024-06-21


In [105]:
forecast = m.predict(future)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2018-06-22,"$8,038.22","$5,233.71","$10,955.94","$8,038.22","$8,038.22",$-0.01,$-0.01,$-0.01,$-0.00,$-0.00,$-0.00,$-0.01,$-0.01,$-0.01,$0.00,$0.00,$0.00,"$7,971.18"
1,2018-06-23,"$8,019.91","$5,164.80","$10,588.40","$8,019.91","$8,019.91",$-0.01,$-0.01,$-0.01,$-0.00,$-0.00,$-0.00,$-0.01,$-0.01,$-0.01,$0.00,$0.00,$0.00,"$7,908.21"
2,2018-06-24,"$8,001.61","$5,400.95","$10,800.83","$8,001.61","$8,001.61",$-0.02,$-0.02,$-0.02,$-0.00,$-0.00,$-0.00,$-0.02,$-0.02,$-0.02,$0.00,$0.00,$0.00,"$7,845.19"
3,2018-06-25,"$7,983.31","$4,749.01","$10,552.79","$7,983.31","$7,983.31",$-0.03,$-0.03,$-0.03,$0.00,$0.00,$0.00,$-0.03,$-0.03,$-0.03,$0.00,$0.00,$0.00,"$7,778.30"
4,2018-06-26,"$7,965.00","$4,894.93","$10,570.24","$7,965.00","$7,965.00",$-0.03,$-0.03,$-0.03,$0.00,$0.00,$0.00,$-0.03,$-0.03,$-0.03,$0.00,$0.00,$0.00,"$7,704.99"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,2024-06-17,"$24,387.23","$-35,024.19","$79,345.66","$-34,501.27","$78,180.54",$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,"$24,750.75"
2188,2024-06-18,"$24,390.70","$-35,220.89","$78,112.15","$-34,776.66","$78,626.24",$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,"$24,686.22"
2189,2024-06-19,"$24,394.18","$-35,162.40","$79,483.34","$-35,037.10","$78,931.31",$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,"$24,574.30"
2190,2024-06-20,"$24,397.65","$-36,705.29","$80,007.60","$-35,270.00","$79,097.17",$0.01,$0.01,$0.01,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$24,524.87"


In [106]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2187,2024-06-17,"$24,750.75","$-35,024.19","$79,345.66"
2188,2024-06-18,"$24,686.22","$-35,220.89","$78,112.15"
2189,2024-06-19,"$24,574.30","$-35,162.40","$79,483.34"
2190,2024-06-20,"$24,524.87","$-36,705.29","$80,007.60"
2191,2024-06-21,"$24,268.93","$-35,723.57","$78,407.56"


In [107]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

22701.403394064695

In [108]:
plot_plotly(m, forecast)

In [109]:
plot_components_plotly(m, forecast)

In [111]:
pickle.dump(m, open('fbprophet.pk1','wb'))